# Regressão Linear Multivariada - Trabalho

## Estudo de caso: Qualidade de Vinhos

Nesta trabalho, treinaremos um modelo de regressão linear usando descendência de gradiente estocástico no conjunto de dados da Qualidade do Vinho. O exemplo pressupõe que uma cópia CSV do conjunto de dados está no diretório de trabalho atual com o nome do arquivo *winequality-white.csv*.

O conjunto de dados de qualidade do vinho envolve a previsão da qualidade dos vinhos brancos em uma escala, com medidas químicas de cada vinho. É um problema de classificação multiclasse, mas também pode ser enquadrado como um problema de regressão. O número de observações para cada classe não é equilibrado. Existem 4.898 observações com 11 variáveis de entrada e 1 variável de saída. Os nomes das variáveis são os seguintes:

1. Fixed acidity.
2. Volatile acidity.
3. Citric acid.
4. Residual sugar.
5. Chlorides.
6. Free sulfur dioxide. 
7. Total sulfur dioxide. 
8. Density.
9. pH.
10. Sulphates.
11. Alcohol.
12. Quality (score between 0 and 10).

O desempenho de referencia de predição do valor médio é um RMSE de aproximadamente 0.148 pontos de qualidade.

Utilize o exemplo apresentado no tutorial e altere-o de forma a carregar os dados e analisar a acurácia de sua solução. 


In [1]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('winequality-white.csv', delimiter=';')
dataset

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.270,0.36,20.70,0.045,45.0,170.0,1.00100,3.00,0.45,8.800000,6
1,6.3,0.300,0.34,1.60,0.049,14.0,132.0,0.99400,3.30,0.49,9.500000,6
2,8.1,0.280,0.40,6.90,0.050,30.0,97.0,0.99510,3.26,0.44,10.100000,6
3,7.2,0.230,0.32,8.50,0.058,47.0,186.0,0.99560,3.19,0.40,9.900000,6
4,7.2,0.230,0.32,8.50,0.058,47.0,186.0,0.99560,3.19,0.40,9.900000,6
5,8.1,0.280,0.40,6.90,0.050,30.0,97.0,0.99510,3.26,0.44,10.100000,6
6,6.2,0.320,0.16,7.00,0.045,30.0,136.0,0.99490,3.18,0.47,9.600000,6
7,7.0,0.270,0.36,20.70,0.045,45.0,170.0,1.00100,3.00,0.45,8.800000,6
8,6.3,0.300,0.34,1.60,0.049,14.0,132.0,0.99400,3.30,0.49,9.500000,6
9,8.1,0.220,0.43,1.50,0.044,28.0,129.0,0.99380,3.22,0.45,11.000000,6


In [33]:
import math
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [34]:
import random
def splitDataset(dataset, splitRatio):
    header = np.asarray(list(dataset))
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    test = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(test))
        trainSet.append(test.pop(index))
    trainSet = pd.DataFrame(trainSet, columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH','sulphates' ,'alcohol', 'quality'])
    test = pd.DataFrame(test, columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH','sulphates' ,'alcohol', 'quality'])
    return [trainSet, test]

In [3]:
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return yhat

In [36]:
splitRatio = 0.7
newDS = dataset.values.tolist()
train, test = splitDataset(newDS, splitRatio)

In [4]:
coef = [0.0 for i in range(len(train.columns))]

In [5]:
for row in range(len(train)):
    yhat = predict(train.iloc[row], coef)
    erro = abs(train.iloc[row][-1] - yhat)
    print("Esperado=%.3f, Predito=%.3f, Erro=%.3f" % (train.iloc[row][-1], yhat, erro))



Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=4.000, Predito=0.000, Erro=4.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Err

Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=8.000, Predito=0.000, Erro=8.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Err

Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=4.000, Predito=0.000, Erro=4.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Err

Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=4.000, Predito=0.000, Erro=4.000
Esperado=8.000, Predito=0.000, Erro=8.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=8.000, Predito=0.000, Erro=8.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=8.000, Predito=0.000, Erro=8.000
Esperado=7.000, Predito=0.000, Err

Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=8.000, Predito=0.000, Erro=8.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Err

Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=4.000, Predito=0.000, Erro=4.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=4.000, Predito=0.000, Erro=4.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=4.000, Predito=0.000, Erro=4.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=4.000, Predito=0.000, Err

Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Err

Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=8.000, Predito=0.000, Erro=8.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=8.000, Predito=0.000, Erro=8.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=8.000, Predito=0.000, Erro=8.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Err

Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=3.000, Predito=0.000, Erro=3.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=4.000, Predito=0.000, Erro=4.000
Esperado=6.000, Predito=0.000, Err

Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=7.000, Predito=0.000, Erro=7.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Erro=6.000
Esperado=5.000, Predito=0.000, Erro=5.000
Esperado=6.000, Predito=0.000, Err

In [6]:
def coefficients_sgd(train, l_rate, n_epoch, coef):
    print ('Coeficiente Inicial={0}' % (coef))
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            sum_error += error**2
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] - l_rate * error * row[i] 
        print(('epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error)))
    return coef

In [27]:
l_rate = 0.01
n_epoch = 20
coef = [0.0 for i in range(len(train.columns))]
newDS = (train - train.min())/(train.max()-train.min())
newTest = (test - test.min())/(test.max()-test.min())
coefs = coefficients_sgd(newDS.values, l_rate, n_epoch, coef)
coefs

Coeficiente Inicial={0}
epoch=0, lrate=0.010, error=70.782
epoch=1, lrate=0.010, error=59.821
epoch=2, lrate=0.010, error=58.336
epoch=3, lrate=0.010, error=57.490
epoch=4, lrate=0.010, error=56.968
epoch=5, lrate=0.010, error=56.635
epoch=6, lrate=0.010, error=56.417
epoch=7, lrate=0.010, error=56.270
epoch=8, lrate=0.010, error=56.170
epoch=9, lrate=0.010, error=56.099
epoch=10, lrate=0.010, error=56.049
epoch=11, lrate=0.010, error=56.012
epoch=12, lrate=0.010, error=55.984
epoch=13, lrate=0.010, error=55.963
epoch=14, lrate=0.010, error=55.946
epoch=15, lrate=0.010, error=55.933
epoch=16, lrate=0.010, error=55.922
epoch=17, lrate=0.010, error=55.912
epoch=18, lrate=0.010, error=55.904
epoch=19, lrate=0.010, error=55.897


[0.3520807627185466,
 -0.09362116196230726,
 -0.3450184709757115,
 0.009030264454627351,
 0.28038140453321675,
 -0.04462653968369987,
 0.1804488443252382,
 -0.033781830521893576,
 -0.022501991467214103,
 0.029986636902992293,
 0.0710487594651131,
 0.3657300530061843]

In [32]:
erro = []
for line in test.values:
    yhat = predict(line, coef)
    erro.append(pow((yhat - line[-1]), 2))  
pow(mean(erro),0.5)

3.0699060821782505